Verificacion de archivo para carga incremental de la tabla review

Preparacion para prueba:
Se utilizara el archivo 'review.json' para sacar tres conjuntos de dataset y se crearan 2 mas:
    1. 'reviews_inicial.json'     , Archivo con la mayoria de la data chunk 14/17     
    2. 'reviews_incremental.json' , Archivo con datos en perfecto estado chunk 2/17
    3. 'reviews_incremental_con_repeticiones.json' , archivo con datos chunk 1/17 y con datos 
        repetidos de chunks anteriores.
    4. 'reviews_incremental_con_fallas_columnas.json', Archivo .json con columnas aleatorio
    5. 'reviews_incremental_con fallas_formato_datos.json', Archivo .json con columnas que coinciden 
        con 'reviews_inicial.json' pero su contenido no.
    
nota: no es necesario tomar en cuenta la fecha del review para clasificar
los datos, es preferible datos aleatorios, los chunks que se crean 
cuando se crea  'reviews_inicial.parquet.gzip'

Procedimiento de verificacion (Bussnes Rules):
Al introducir un nuevo archivo .json se verifica en formato:
    1. cantidad de columnas
    2. nombre de columnas
    3. formato de datos en columnas
Si cumple con formato se verifica contenido, se trabaja con los codigos alfanumericos 
del nuevo dato/review review_id,user_id y business_id:

    4. tabla 'idreview[id_review(int),review_id(tex)]';
        4.1. si el review_id ya existen en tabla base 
            se descarta el dato/review nuevo
        4.2. de lo contrario
            se revisan los puntos 5.1 y 6.1
            se incorpora el nuevo review a la tabla reviews y a la tabla idreviews

    5. tabla idusers[id_user(int),user_id(tex)], 
        5.1 si el user_id NO esta en la tabla 'idusers' 
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo user a la tabla 'users' para lo cual se necesitarian los datos [review_count, yelping_since, .... average_stars], al actualizar esa data automatiacamente se actualiza la tabla 'idusers' y despues se procede con el punto 4.2


    6. tabla 'idbusiness[id_business(int),business_id(tex)], 
        6.1 si el business_id del nuevo data/review NO esta en la tabla 'idbusiness'
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo busines a la tabla 'business' para lo cual se necesitarian los datos [name, address, .... categories_id], adicionalmente se debe verifiar su categoria,
    si coincide con las categorias a usar se agrega a la tabla  'business', y automaticamente actualizar las tablas 'idbusiness', 'business_attributes', 'business_categories', 'business_city_state', 'business_hours' para luego proceder con el punto 4.2


Reqerimientos:
    1. Se requiere acceder a la tabla idreviews 
    2. Se requiere  acceder a la tabla idbussines 
    3. Se requiere  acceder a la tabla idusers

### Condiciones de incio

In [1]:
#Librerias a utilizar
import numpy 
import pandas
import pandas as pd
import numpy as np
from IPython.display import clear_output
import pymysql


In [3]:
#Direccion incial donde estan archivos de entrada, y direccion de archivos salida
path_incial = 'C:\\Github\\DataSet_YELP\\'
path_final = 'C:\\Github\\DataSet_YELP\\pruebas_incremental\\'

### Creacion de datos de prueba

In [4]:


cont = 0
#Se sabe que tamaño +200k  se haran 
# 35 partes (chunk) del .json este 
# estimado es solo para "reviews.json"
cont_max = 35  
Tamano = 200000
for chunk in pd.read_json(path_incial+'review.json',chunksize=Tamano, lines=True):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado carga: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Se carga cada parte
    if cont == 1:
        review = chunk
    else:
        review = pd.concat([review, chunk], ignore_index=True)
        
#tiempo = 7 min

Completado carga: 35/35  100.0%


In [ ]:
#Arbitrariamente se define:  
#   70% de data en 'reviews_inicial.json' , 
#   20% en 'reviews_incremental.json' y 
#   10% en 'reviews_incremental_con_repeticiones.json'

#creacion de archivo 'reviews_inicial.json' 
corte_0 = int(round(review.shape[0]*0.7,0))
reviews_inicial = review.iloc[:corte_0,:]
reviews_inicial.to_json(path_final+'reviews_inicial.json')

#creacion de archivo 'reviews_incremental.json' 
corte_1 = int(round(review.shape[0]*0.9,0))
reviews_incremental = review.iloc[corte_0+1:corte_1,:]
reviews_incremental.to_json(path_final+'reviews_incremental.json')

#creacion de archivo 'reviews_incremental_con_repeticiones.json' 
df = review.iloc[corte_1+1:,:]
df2 = reviews_incremental.sample(200000)
review_repeticiones = pd.concat([df, df2], ignore_index=True)
review_repeticiones.to_json(path_final+'reviews_incremental_con_repeticiones.json')

#creacion de archivo .json con columnas aleatorio
df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
df.to_json(path_final+'reviews_aleatorio1.json')

#creacion de archivo .json con columnas correctas pero datos aleatorio
columnas = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny','cool', 'text', 'date']
df = pd.DataFrame(np.random.randn(1000, 9), columns=columnas)
df.to_json(path_final+'reviews_aleatorio2.json')

del review, columnas, corte_1, corte_0, df, df2, reviews_inicial, reviews_incremental, review_repeticiones

### Procedimiento de verificacion

In [5]:
#archivo_en_verificacion = 'reviews_aleatorio2.json'
#archivo_en_verificacion = 'reviews_aleatorio1.json'
#archivo_en_verificacion = 'reviews_incremental.json'
archivo_en_verificacion =  'reviews_incremental_con_repeticiones.json'
path = 'C:\\Github\\DataSet_YELP\\pruebas_incremental\\'

In [44]:
review = pd.read_json(path+archivo_en_verificacion)

#Mejor seria cargar inicialmente la primera fila y 
# despues todo el archivo pero como son pequeños no hay problema en hacerlo al reves 
review_prueba = df.iloc[0:1,:]

In [67]:
#condicion incial
check_03_Columnas = False

# revision cantidad columnas
check_01_Columnas = len(review_prueba.columns) == 9

#nombre de columnas
if check_01_Columnas:
    L1 = ['review_id', 'user_id', 'business_id', 'useful','stars',  'funny','cool', 'text', 'date']
    L2 = list(review_prueba.columns)
    check_02_Columnas = len([i for i in L1 if i in L2]) == 9

#formato de datos en columnas
if check_02_Columnas:
    def chequea_formato_columnas(review_prueba):
        resultado = True
        resultado = isinstance(review_prueba['review_id'].loc[0], str) & resultado
        resultado = isinstance(review_prueba['user_id'].loc[0], str) & resultado
        resultado = isinstance(review_prueba['business_id'].loc[0], str) & resultado
        resultado = isinstance(review_prueba['stars'].loc[0], numpy.int64) & resultado
        resultado = isinstance(review_prueba['useful'].loc[0], numpy.int64) & resultado
        resultado = isinstance(review_prueba['funny'].loc[0], numpy.int64) & resultado
        resultado = isinstance(review_prueba['text'].loc[0], str) & resultado
        resultado = isinstance(review_prueba['date'].loc[0], pandas._libs.tslibs.timestamps.Timestamp)  & resultado
        return resultado
    check_03_Columnas = chequea_formato_columnas(review_prueba)

In [69]:
import pymysql

In [72]:
#Conexion para carga de tablas Id desde  base de datos local
conexion = pymysql.connect(
   host = '127.0.0.1',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='proyecto_yelp_24_11_22'
)
cursor = conexion.cursor()

#conexion a googlecloud
conexionGooglecloud = pymysql.connect(
   #host = '177.226.145.197',  #para revisar si conecta
   host = '34.176.218.33',
   user = 'root',
   passwd='projectyelp2022',
   db='prueba'
)
cursorcloud = conexionGooglecloud.cursor()

In [73]:
cursor.execute("show tables;")

Resultado = cursor.fetchall()

In [74]:
Resultado

(('business',),
 ('business_attributes',),
 ('business_categories',),
 ('business_city_state',),
 ('business_hours',),
 ('idbusiness',),
 ('iduser',),
 ('reviews',),
 ('reviews2',),
 ('users',))

In [ ]:
if check_03_Columnas:
    #Carga de tabla id reviews desde la base de datos
    
    
    reviews_inicial.parquet.gzip
    pass


    


Procedimiento de verificacion (Bussnes Rules):
Al introducir un nuevo archivo .json se verifica en formato:
    1. cantidad de columnas
    2. nombre de columnas
    3. formato de datos en columnas
Si cumple con formato se verifica contenido, se trabaja con los codigos alfanumericos 
del nuevo dato/review review_id,user_id y business_id:

    4. tabla 'idreview[id_review(int),review_id(tex)]';
        4.1. si el review_id ya existen en tabla base 
            se descarta el dato/review nuevo
        4.2. de lo contrario
            se revisan los puntos 5.1 y 6.1
            se incorpora el nuevo review a la tabla reviews y a la tabla idreviews

    5. tabla idusers[id_user(int),user_id(tex)], 
        5.1 si el user_id NO esta en la tabla 'idusers' 
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo user a la tabla 'users' para lo cual se necesitarian los datos [review_count, yelping_since, .... average_stars], al actualizar esa data automatiacamente se actualiza la tabla 'idusers' y despues se procede con el punto 4.2


    6. tabla 'idbusiness[id_business(int),business_id(tex)], 
        6.1 si el business_id del nuevo data/review NO esta en la tabla 'idbusiness'
            Se descarta dato/review nuevo
    nota: un procedimento mas correcto seria, agrega el nuevo busines a la tabla 'business' para lo cual se necesitarian los datos [name, address, .... categories_id], adicionalmente se debe verifiar su categoria,
    si coincide con las categorias a usar se agrega a la tabla  'business', y automaticamente actualizar las tablas 'idbusiness', 'business_attributes', 'business_categories', 'business_city_state', 'business_hours' para luego proceder con el punto 4.2